In [1]:




import matplotlib.pyplot as plt
import random

import os

import datasets
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader, random_split

from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import VisionEncoderDecoderModel , ViTFeatureExtractor
from transformers import AutoTokenizer ,  GPT2Config , default_data_collator


if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

%autosave 1

No GPU available, using the CPU instead.


Autosaving every 1 seconds


In [2]:
notebook_name = None

In [3]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "notebook_name = " + "'"+thename+"'";
# kernel.execute(command);


In [5]:
if notebook_name is None:
    notebook_name = 'multi_modal.ipynb'
print(notebook_name)

multi_modal.ipynb


In [6]:
# 

In [7]:
from time import gmtime, strftime
session_key = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(session_key)

2023-04-02-16-15-50


In [8]:
import wandb

os.environ['WANDB_NOTEBOOK_NAME'] = notebook_name

wandb.init(project="component-properties-generation",
        config={
            "learning_rate": None,
            "epochs":  None,
            "batch_size":  None,
            "val_split":  None,
            "dropout":  None,
            "image_width":  None,
            "image_height":  None,
            "random_seed":  None,
            "session_key":  None,
            "model_name": "ResNet50V2",
            "train_accuracy": None,
            "val_accuracy": None,
            "test_accuracy": None,
            "class_weight_strategy": None,
            "fine_tuning": None,
            "pad_input": None,
            "pad_strategy":None
        })

w_config = wandb.config



wandb: Currently logged in as: vamsidhar_muthireddy. Use `wandb login --relogin` to force relogin


In [9]:
w_config.learning_rate = 0.001
w_config.epochs = 2
w_config.batch_size = 32
w_config.val_split = 0.2
w_config.dropout = 0.1
w_config.image_width = 128
w_config.image_height = 128
w_config.random_seed = 42
w_config.session_key = session_key
w_config.model_name = "vit-gpt2"
w_config.train_accuracy = 0
w_config.val_accuracy = 0
w_config.test_accuracy = 0
w_config.class_weight_strategy = "Augmented"    #keys: None, "Original", "Augmented"
w_config.fine_tuning = False                     #Fine-tunes the model if True
w_config.pad_input = True
w_config.pad_strategy = 'pad_only'      #keys: 'pad_only', 'scale_and_pad'

Organizing data for model training and inferencing



Model training will happen on images from train_folder <br>
train_folder = Training set (Will be split into train and validation to be used for model training)

Model inferencing will happen on images from test_folder <br>
test_folder = Test set (Will be used to evaluate the trained model for <b>inference</b>)



In [10]:

shuffle_dataset = True



save_folder = '/home/vamsidhar/Documents/purecode/solution_1/test_' + str(w_config.session_key)
print(save_folder)
os.makedirs(save_folder, exist_ok=True)

model_save_path = os.path.join(save_folder, "model_weights.h5")
architecture_save_path = os.path.join(save_folder, "model_architecture.txt")


np.random.seed(w_config.random_seed)
random.seed(w_config.random_seed)



train_df_path = '/home/vamsidhar/Documents/purecode/solution_1/df_train.pkl'
test_df_path = '/home/vamsidhar/Documents/purecode/solution_1/df_test.pkl'

train_folder = '/home/vamsidhar/Documents/purecode/data_3/train/images'
test_folder = '/home/vamsidhar/Documents/purecode/data_3/test/images'

tokenizer_path = '/home/vamsidhar/Documents/purecode/solution_1/tokenizer.pkl'



/home/vamsidhar/Documents/purecode/solution_1/test_2023-04-02-16-15-50


In [11]:
import joblib

df_train = joblib.load(train_df_path)
df_test = joblib.load(test_df_path)



In [12]:


def get_weights(df):
    cls = df.comp_label.unique()
    
    cls_count = df.comp_label.value_counts()
    maximum = max(cls_count)
    cls_count = maximum/cls_count
    
    weights = cls_count.to_dict()


    return weights





In [14]:

classes_train = df_train.comp_type.unique()
classes_test  = df_test.comp_type.unique()
print(classes_train)

num_classes = len(classes_train)


class_weights = get_weights(df=df_train)
    
print(class_weights)


['Switch' 'Checkbox' 'Card' 'Icon' 'Button']
{3: 1.0, 4: 1.758058399696625, 2: 2.136405529953917, 1: 3.6049766718506997, 0: 14.578616352201259}


In [15]:
   


def generate_filenames_label(df, shuffle=True, split=False, split_ratio=w_config.val_split):
    if not split:
        file_name_img = df.path_images.values
        properties = list(df.text.values)
#         label = label_encoding(df)
        label = df.comp_label.values
    
        return label, properties, file_name_img
    
    else:
        df_temp = df.copy()
        
        df_val = df_temp.sample(frac = 0.2, random_state=42)
        df_train = df_temp.drop(df_val.index)

        df_val.reset_index(inplace=True)
        df_train.reset_index(inplace=True)

        file_name_img_train = df_train.path_images.values
        properties_train = list(df_train.text.values)
#         label_train = label_encoding(df_train)
        label_train = df_train.comp_label.values
        
        
        file_name_img_val = df_val.path_images.values
        properties_val = list(df_val.text.values)
#         label_val = label_encoding(df_val)
        label_val = df_val.comp_label.values
        
        
        return label_train, properties_train, file_name_img_train, label_val, properties_val, file_name_img_val  


    
    

In [16]:
label_train, prop_train, file_name_train, label_val, prop_val, file_name_val = generate_filenames_label(df_train,
                                                                                  shuffle=True,
                                                                                  split=True,
                                                                                  split_ratio=w_config.val_split)



In [17]:
label_test, prop_test, file_name_test  = generate_filenames_label(df_test, shuffle=False, split=False)



In [18]:
from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import TFVisionEncoderDecoderModel , ViTFeatureExtractor
from transformers import AutoTokenizer ,  GPT2Config , default_data_collator

import multiprocessing as mp

In [19]:
class config : 
    ENCODER = "google/vit-base-patch16-224"
    DECODER = "gpt2"
    TRAIN_BATCH_SIZE = 8
    VAL_BATCH_SIZE = 8
    VAL_EPOCHS = 1
    LR = 5e-5
    SEED = 42
    MAX_LEN = 350
    SUMMARY_LEN = 20
    WEIGHT_DECAY = 0.01
    MEAN = (0.485, 0.456, 0.406)
    STD = (0.229, 0.224, 0.225)
    TRAIN_PCT = 0.95
    NUM_WORKERS = mp.cpu_count()
    EPOCHS = 3
    IMG_SIZE = (224,224)
    LABEL_MASK = -100
    TOP_K = 1000
    TOP_P = 0.95

In [20]:
from transformers import AutoTokenizer ,  GPT2Config , default_data_collator

def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs
AutoTokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens




In [21]:
import datasets

rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

/tmp/ipykernel_412307/3810674018.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")


In [22]:
feature_extractor = ViTFeatureExtractor.from_pretrained(config.ENCODER)
tokenizer = AutoTokenizer.from_pretrained(config.DECODER)
tokenizer.pad_token = tokenizer.unk_token

# tokenizer = joblib.load(tokenizer_path)

/home/vamsidhar/miniconda3/envs/purecode_cpu/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [23]:
# import cv2
# img = cv2.imread(df_train.path_images[0])/255

# feature_extractor([img, img, img], return_tensors='tf').pixel_values


In [40]:
class ImgDataset(Dataset):
    def __init__(self, filenames, labels, props, tokenizer,
                 feature_extractor, shuffle=True):
        
        self.labels = labels
        self.filenames = filenames
        self.props = props
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        
        
        
#         self.df = df
#         self.transform = transform
#         self.root_dir = root_dir
        self.max_length = 350
        
    def __len__(self,):
        return len(self.filenames)
    
    def __getitem__(self,idx):
        caption = self.props[idx]
        img_path = self.filenames[idx]
#         img = self.__load_image__(img_path)
        img = Image.open(img_path).convert("RGB")
        
#         if self.transform is not None:
#             img= self.transform(img)
        pixel_values = self.feature_extractor(img, return_tensors="pt").pixel_values
        captions = self.tokenizer(caption,
                                 padding='max_length',
                                  truncation=True,
                                 max_length=self.max_length).input_ids
        
        captions = [caption if caption != self.tokenizer.pad_token_id else -100 for caption in captions]
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(captions)}
        return encoding
    
    
#     def __load_image__(self, image_path):
#         img = cv2.imread(image_path)
#         img = self.__resize_image__(img, pad=True)
            
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = img.astype(np.float32) / 255.0
        
        
#         return img

#     def __resize_image__(self, img, pad=True):
#         old_size = img.shape[:2]
#         ratio = float(self.dim[0])/max(old_size)
#         new_size = tuple([int(x*ratio) for x in old_size])
#         img_new = cv2.resize(img, (new_size[1], new_size[0])) 

#         #resize by padding
#         delta_w = self.dim[1] - new_size[1]
#         delta_h = self.dim[0] - new_size[0]
#         top, bottom = delta_h//2, delta_h-(delta_h//2)
#         left, right = delta_w//2, delta_w-(delta_w//2)

#         color = [0, 0, 0]
#         img_new = cv2.copyMakeBorder(img_new, top, bottom, left, right, cv2.BORDER_CONSTANT,
#             value=color)

        
#         return img_new



In [41]:
train_dataset = ImgDataset(filenames=file_name_train, labels=label_train, props=prop_train,
                           tokenizer=tokenizer,feature_extractor = feature_extractor )

val_dataset = ImgDataset(filenames=file_name_val, labels=label_val, props=prop_val,
                           tokenizer=tokenizer,feature_extractor = feature_extractor )

test_dataset = ImgDataset(filenames=file_name_test, labels=label_val, props=prop_test,
                           tokenizer=tokenizer,feature_extractor = feature_extractor )





In [42]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(config.ENCODER, 
                                                                  config.DECODER)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly ini

In [43]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size
# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.max_length = 350
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [44]:
training_args = Seq2SeqTrainingArguments(
    output_dir='VIT_large_gpt2',
    per_device_train_batch_size=config.TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=config.VAL_BATCH_SIZE,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=1024,  
    save_steps=2048, 
    warmup_steps=1024,  
    learning_rate = 5e-5,
    #max_steps=1500, # delete for full training
    num_train_epochs = config.EPOCHS, #TRAIN_EPOCHS
    overwrite_output_dir=True,
    save_total_limit=1,
)

In [45]:
# import torch
# torch.cuda.empty_cache()

In [46]:
from transformers import default_data_collator
import cv2

trainer = Seq2SeqTrainer(
    tokenizer=feature_extractor,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
